## Creation of the dataset

We have two datasets
 - Amazon Customer Review dataset :
   - amazon_reviews_us_Books_v1_00.tsv.gz (2.74 Go)
   - amazon_reviews_us_Books_v1_01.tsv.gz (2.69 Go)
   - amazon_reviews_us_Books_v1_02.tsv.gz (1.33 Go)
 - Amazon Book Cover dataset : 
   - book32-listing.csv (39.5 Mo)

Each row of the first dataset represents a review. Therefore we could have several rows representing the same book (several reviews for the same book). 

Each row of the second dataset represents a book. 

In this notebook, we merge the two datasets, in order to have each row of the new dataset representing a review, along with the cover of the corresponding book. 

Since the first dataset is pretty big, and to avoid creating a huge dataset, we will use the following pipeline : 
 - Load the cover file
 - Load the first review file
 - Merge the first review file in df_merge_1
 - Load the second review file
 - Merge the second review file in df_merge_2
 - Load the third review file
 - Merge the third review file in df_merge_3
 - Merge df_merge_1, df_merge_2, and df_merge_3 in df_merge
 - Save df_merge in merge.csv

In [5]:
import pandas as pd
import numpy as np

First we load the cover image dataset

In [6]:
columns = ['product_id', 
           'filename', 
           'image_url', 
           'title', 
           'author', 
           'category_id', 
           'category']
df_covers = pd.read_csv("Data/book32-listing.csv", 
                        encoding = "ISO-8859-1", 
                        header=None, 
                        names=columns)
df_covers.head()

,product_id,filename,image_url,title,author,category_id,category
0,761183272,0761183272.jpg,http://ecx.images-amazon.com/images/I/61Y5cOdH...,Mom's Family Wall Calendar 2016,Sandra Boynton,3,Calendars
1,1623439671,1623439671.jpg,http://ecx.images-amazon.com/images/I/61t-hrSw...,Doug the Pug 2016 Wall Calendar,Doug the Pug,3,Calendars
2,B00O80WC6I,B00O80WC6I.jpg,http://ecx.images-amazon.com/images/I/41X-KQqs...,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",Moleskine,3,Calendars
3,761182187,0761182187.jpg,http://ecx.images-amazon.com/images/I/61j-4gxJ...,365 Cats Color Page-A-Day Calendar 2016,Workman Publishing,3,Calendars
4,1578052084,1578052084.jpg,http://ecx.images-amazon.com/images/I/51Ry4Tsq...,Sierra Club Engagement Calendar 2016,Sierra Club,3,Calendars


We read the dataset containing the reviews. We have to specify the columns names because some rows have a higher count of cell than others. We want to see what is contained in these rows. 

In [7]:
cols = ["marketplace",
        "customer_id",
        "review_id",
        "product_id",
        "product_parent",
        "product_title",
        "product_category",
        "star_rating",
        "helpful_votes",
        "total_votes",
        "vine",
        "verified_purchase",
        "review_headline",
        "review_body",
        "review_date", 
        "undefined_1",
        "undefined_2",
        "undefined_3",
        "undefined_4",
        "undefined_5",
        "undefined_6",
        "undefined_7"
       ]

In [8]:
df_reviews = pd.read_csv("Data/amazon_reviews_us_Books_v1_00.tsv.gz", 
                         names=cols, 
                         sep='\t', 
                         skiprows=1)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The warning simply indicates that some columns do not have the same dtype for each row, this will be handled later by cleaning the data.

So first we check the rows with a higher count of cells.

In [9]:
df_reviews[df_reviews["undefined_1"].notnull()]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,review_headline,review_body,review_date,undefined_1,undefined_2,undefined_3,undefined_4,undefined_5,undefined_6,undefined_7
3522,US,45223684,R1NHKVD5LMP2JY,1503551814,353548609,A Nutty World: On the Edge of the Rain Forest,Books,5,0.0,0.0,...,She is a fine storyteller and has a message c...,Books,5,0.0,0.0,N,Y,looks great.,Very durable. looks great.,2015-08-31
5280,US,18701079,R1946J1OJKC1K2,0743292472,7685359,The Ultimate Battle: Okinawa 1945--The Last Ep...,Books,5,0.0,0.0,...,was a great read. Quick-moving with sharp int...,Books,1,6.0,7.0,N,Y,NOT PLEASED WITH THE VENDORS AT ALL,THE CASE IS MADE OF A CHEAP QUALITY. IT BROKE ...,2015-08-31
20476,US,40945620,R33MH6R1TZWIGR,1452572747,523136939,"Have This Mind: Supreme Happiness, Ultimate Re...",Books,5,1.0,1.0,...,This wonderful book is thoughtful and well-wri...,Books,5,0.0,0.0,N,Y,Very good!,Everything as promised. Very good!,2015-08-30
25893,US,22921197,R1OPQAX1D7DWB8,0984755101,632337521,The Primal Connection: Follow Your Genetic Blu...,Books,5,0.0,0.0,...,this is the best book on health and wellness ...,Books,4,1.0,2.0,N,N,Don't Be Mislead...,This is not so much intended to be a review as...,2015-08-29
27014,US,44087253,R24H2K78E883A0,1616496169,709191612,Navigating Grace: A Solo Voyage of Survival an...,Books,5,2.0,2.0,...,"Oh would some Power the gift give us, To ...\t...",Books,5,0.0,0.0,N,Y,It was a great experience and one I will alway...,After reading the book I visited Helen and too...,2015-08-29
59796,US,44250036,RYNEKW5P1Z3AG,0385262078,887509044,The Impossible H L. Mencken,Books,5,0.0,0.0,...,One of the most important of all American men...,Books,5,0.0,0.0,N,Y,The storyline is great and the characters are ...,This book is the third in the series and in my...,2015-08-26
69196,US,50641462,R2VI5KEAQR2P23,0545436826,247155770,Friendshape,Books,3,6.0,6.0,...,It is a bunch of sweet musings about friendsh...,Books,5,0.0,0.0,N,N,... have heard a few things about Medjugorje b...,I have heard a few things about Medjugorje but...,2015-08-25
71951,US,43676679,R2F0SUDYHR79JQ,1844094642,766424526,"Change Your Story, Change Your Life: Using Sha...",Books,2,4.0,16.0,...,I intensely dislike the psychology perspectiv...,Books,5,1.0,2.0,N,Y,Five Stars,love it,2015-08-25
78718,US,4881222,R21NUSCF1WC1I8,1607512572,236705009,The House on Tradd Street,Books,4,0.0,0.0,...,discriptive of characters w/repetitive plot di...,Books,5,7.0,8.0,N,N,I like Kevin Trudeau - great information yet ...,I like Kevin Trudeau because he has made it hi...,2015-08-24
81298,US,1148395,R1UH3R71GX97AR,1502716984,563128335,Jack The Ripper: Newly Discovered Adventures o...,Books,1,1.0,2.0,...,is a complete waste of time and money\tthis &...,Books,5,0.0,0.0,N,Y,Five Stars,This item was received. Thank you.,2015-08-24


In [10]:
# Compute the proportion of long row
null_mask = df_reviews["undefined_1"].notnull()
print(null_mask.sum() )
print((~null_mask).sum())

326
10236850


What is contained in the undefined columns is a bit weird, it's like we have more reviews, but they are not really related to the standard review column. For example, in the second line of the output, the review is "Was a great read..." and then we have "NOT PLEASED WITH THE VENDORS AT ALL" etc.

These lines are not clear concerning the reviews. We also try to find these reviews on the amazon website. We found some of them, but we were only able to find the main reviews, and not the reviews contained in the additional columns. Besides, these rows are not coherent concerning the columns (some values represents something else than expected by the column name). Since we only have 326 of such rows over 10 millions, we decide to drop these rows. We can also drop the additional column since we won't need them anymore. 

Finally, we drop some other columns that we don't need 

In [11]:
df_reviews = df_reviews[~null_mask]
df_reviews = df_reviews.iloc[:, :15]
df_reviews = df_reviews.drop(['product_parent', 
                              'product_category', 
                              'vine', 
                              'verified_purchase'], axis=1)

In [12]:
df_reviews["product_id"] = df_reviews["product_id"].astype('str')

Now we want to merge the reviews dataset with the covers dataset, in other word, for each reviews, we want to have the cover image of the book. 

In [13]:
df_merge_1 = df_reviews.merge(df_covers, 
                              left_on='product_id', 
                              right_on='product_id', 
                              how='inner')

In [14]:
print(len(df_merge_1))

1409974


From the 10M reviews, we have 1.4M that corresponds to a book from the 137'000 books of the covers dataset. The loss is consequent, but we still have a important dataset to work on. Moreover, we still have the two others reviews dataset to include in this dataframe.

Note that we prefered not to load all three reviews dataset because of memory concerns. Our pipeline is : load covers, load review_1, merge, save merge as csv, load review_2, merge, save merge as csv, load review_3, merge, save merge as csv. 

In [15]:
df_reviews = pd.read_csv("Data/amazon_reviews_us_Books_v1_01.tsv.gz", 
                         names=cols, 
                         sep='\t', 
                         skiprows=1)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
null_mask = df_reviews["undefined_1"].notnull()
print(null_mask.sum() )
print((~null_mask).sum())

1
6106190


In [17]:
df_reviews = df_reviews[~null_mask]
df_reviews = df_reviews.iloc[:, :15]
df_reviews = df_reviews.drop(['product_parent', 
                              'product_category', 
                              'vine', 
                              'verified_purchase'], axis=1)

In [18]:
df_reviews["product_id"] = df_reviews["product_id"].astype('str')

In [19]:
df_merge_2 = df_reviews.merge(df_covers, 
                              left_on='product_id', 
                              right_on='product_id', 
                              how='inner')

In [20]:
print(len(df_merge_2))

359693


Finally the third reviews dataset

In [21]:
df_reviews = pd.read_csv("Data/amazon_reviews_us_Books_v1_02.tsv.gz", 
                         names=cols, 
                         sep='\t', 
                         skiprows=1)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
null_mask = df_reviews["undefined_1"].notnull()
print(null_mask.sum() )
print((~null_mask).sum())

1
3105370


In [23]:
df_reviews = df_reviews[~null_mask]
df_reviews = df_reviews.iloc[:, :15]
df_reviews = df_reviews.drop(['product_parent', 
                              'product_category', 
                              'vine', 
                              'verified_purchase'], axis=1)

In [24]:
df_reviews["product_id"] = df_reviews["product_id"].astype('str')

In [25]:
df_merge_3 = df_reviews.merge(df_covers, 
                            left_on='product_id', 
                            right_on='product_id', 
                            how='inner')

In [26]:
print(len(df_merge_3))

72122


In [27]:
# append the dataframes
df_merge = df_merge_1.append(df_merge_2).append(df_merge_3)

In [28]:
df_merge.to_csv("Data/merge.csv", sep='\t', index=False)